In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd
import pickle

import spacy

max_length = 20

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
len(vocab)
# vocab.get('->', -1), vocab

69256

In [3]:
vocab.get('<---"<user>', -1)

65903

# Load Embeddings

In [4]:
print("loading embeddings")
#     with open('cooc.pkl', 'rb') as f:
#         cooc = pickle.load(f)
embeddings = np.load('embeddings.npy')

loading embeddings


In [5]:
embeddings.shape

(69256, 20)

# Loading training set

In [4]:
#Read Data neg and pos with corresponding label

raw_data_neg = pd.read_csv('train_neg_full.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
# raw_data_neg['text'] = raw_data_neg['text']
raw_data_neg['label'] = 0
# raw_data_neg = raw_data_neg[:5000]

raw_data_pos = pd.read_csv('train_pos_full.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
raw_data_pos['label'] = 1
# raw_data_pos = raw_data_pos[:5000]


raw_data_train = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)


raw_data_test = pd.read_csv('test_data.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()


raw_data_test['id'], raw_data_test['text'] = raw_data_test['text'].apply(lambda x: x.split(',')[0]), raw_data_test['text'].apply(lambda x: ','.join(x.split(',')[1:]))

In [5]:
docs = list(nlp.pipe(raw_data_train['text'], batch_size=1000, n_threads=8))

# test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))

In [12]:
test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))

In [14]:
len(test_docs)

with open('test_docs_saved', 'wb') as f:
    pickle.dump(test_docs, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('test_docs_saved', 'rb') as f:
        test_docs_loaded = pickle.load(f)


In [22]:
with open('train_docs_saved', 'wb') as f:
    pickle.dump(docs, f, pickle.HIGHEST_PROTOCOL)

In [21]:
len(test_docs_loaded), len(test_docs)

(10000, 10000)

In [23]:
# nlp(raw_data_train.text[0]).vector
# nlp(raw_data_train.text)

# docs = list(nlp.pipe(raw_data_train['text'], batch_size=1000, n_threads=8))

# test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))
# print(max([len(x) for x in docs]))

# np.zeros((raw_data_test['processed_text'].shape[0], max_length))

features_train = np.zeros((len(docs), 300))

for (i, doc) in enumerate(docs):
    features_train[i, :] = doc.vector
    
features_train



features_test = np.zeros((len(test_docs), 300))

for (i, doc) in enumerate(test_docs):
    features_test[i, :] = doc.vector
    
    
features_train.shape, features_test.shape

((2270482, 300), (10000, 300))

In [24]:
X_train, y_train = features_train, raw_data_train.label
X_test = features_test

In [35]:
X_train.shape, y_train.shape, X_test.shape

((2270482, 300), (2270482,), (10000, 300))

# Construct Features for the Training Texts
convert each word in each tweet to a number (which indicates how frequent it is)

In [59]:
# raw_data.text[0].strip().split()
# raw_data_train['processed_text'] = raw_data_train.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))
# raw_data_test['processed_text'] = raw_data_test.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))

raw_data_train['processed_text'] = raw_data_train.text.apply(lambda x: x.strip()).apply(lambda x: nlp(x))
raw_data_test['processed_text'] = raw_data_test.text.apply(lambda x: x.strip()).apply(lambda x: nlp(x))


raw_data_train.processed_text
raw_data_test.processed_text

KeyboardInterrupt: 

Average word vectors over all words of the tweet

In [9]:
raw_data_train.processed_text[0], raw_data_test.processed_text[0]

(array([  552,    76, 16312,   466,    17,   466, 12655,    62,     9,
            7, 20052,  3368,    33,   170,  8252,    23,    25]),
 array([ 2653,  2824,  2265,  2653,  8695,    76,  2070,    26,     7,
        17085,  1130,     6,   378,  1318,    16,     7,   786,     9,
           23,    25]))

In [10]:
# raw_data_test[raw_data_test['id']== 1]
raw_data_test.loc[4588]

text              <user> its freezing haha bas helw helw .. ta3a...
id                                                             4589
processed_text    [0, 85, 5862, 64, 8497, 39, 4, 60, 37, 3, 93, ...
Name: 4588, dtype: object

In [24]:
 embeddings.mean(0).reshape(1, -1).shape

(1, 20)

In [33]:
features_train = np.zeros((raw_data_train['processed_text'].shape[0], max_length))
average_word_vectors_train = raw_data_train['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else embeddings.mean(0).reshape(1, -1)).apply(lambda x: x.mean(0))

for (i, doc) in enumerate(average_word_vectors_train):
    features_train[i, :doc.shape[0]] = doc


X_train, y_train = features_train, raw_data_train.label


features_test = np.zeros((raw_data_test['processed_text'].shape[0], max_length))
# average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else np.zeros(max_length)).apply(lambda x: x.mean(0))
average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else embeddings.mean(0).reshape(1, -1)).apply(lambda x: x.mean(0))

# for (i, asd) in enumerate(raw_data_test['processed_text']):
#     print("i: ", i)
#     print(embeddings[asd, :], asd.shape)



for (i, doc) in enumerate(average_word_vectors_test):
#     print("i: ", i, doc)
    features_test[i, :doc.shape[0]] = doc

average_word_vectors_test

X_test = features_test

In [34]:
X_train.shape, y_train.shape, embeddings.shape, X_test.shape

NameError: name 'embeddings' is not defined

# Train a Linear Classifier
Let's start with logistic regression

In [46]:
from sklearn import svm

model = LogisticRegression(random_state=0, solver='sag', multi_class='ovr')
# model = svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear')
# model = svm.LinearSVC(dual=False)


fitted_model = model.fit(X_train, y_train)

# Prediction
Predict labels for all tweets in the test set.

In [27]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
#             print(y_pred)
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})


In [40]:
y_test = fitted_model.predict(X_test)

In [41]:
y_train_predict = fitted_model.predict(X_train)

In [42]:
np.sum((y_train == y_train_predict)) / y_train.shape[0]

0.7747337349514333

In [43]:
y_test[y_test==0] = -1

In [44]:
y_test

array([-1,  1, -1, ..., -1,  1, -1])

In [45]:
create_csv_submission(raw_data_test['id'], y_test, 'logistic_regression_submission_2_full_dataset.csv')